In [1]:
import pandas as pd

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['No'].tolist()

总数量：2


[1, 2]

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import json

from bs4 import BeautifulSoup
from lxml import etree

url = input_.loc[a, 'Url']

dict_param = {}
list_param = url.split('?')[1].split('&') if '?' in url else []
for param in list_param:
    dict_param[param.split('=')[0]] = param.split('=')[1]

dict_param.pop('_pgn', '')
dict_param['orig_cvip'] = 'true'
dict_param['_ipg'] = '240'

if url.startswith('https://www.ebay.com/'):
    dict_param['_ul'] = 'US'
    dict_param['_stpos'] = '91710'
elif url.startswith('https://www.ebay.de/'):
    dict_param['_ul'] = 'DE'
    dict_param['_stpos'] = '10115'
elif url.startswith('https://www.ebay.co.uk/'):
    dict_param['_ul'] = 'GB'
    dict_param['_stpos'] = 'BS81QU'
elif url.startswith('https://www.ebay.com.au/'):
    dict_param['_ul'] = 'AU'
    dict_param['_stpos'] = '2019'
elif url.startswith('https://www.ebay.ca/'):
    dict_param['_ul'] = 'CA'
    dict_param['_stpos'] = 'M5S2E8'

url = url.split('?')[0] + '?'
for key, value in dict_param.items():
    url += f'{key}={value}&'

list_item_number, page = [], 1
while True:
    b = 0
    while True:
        b += 1
        print(b, page)
        try:
            request_url = f'''{url}_pgn={page}'''
            
            resp = requests.get(request_url,
                                headers=get_header(ua=False),
                                proxies=get_proxy(),
                                timeout=(10, 10))
            
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0].strip() + ']}]}')

                list_item_number_temp = [w[2]['model']['action']['params']['itemId'].strip() if type(w[2]) == dict and 'model' in w[2] and 'action' in w[2]['model'] and 'params' in w[2]['model']['action'] and 'itemId' in w[2]['model']['action']['params'] else '' for w in dict_['o']['w']]
                if list_item_number_temp:
                    while '' in list_item_number_temp:
                        list_item_number_temp.remove('')
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    list_item_number += list_item_number_temp
    list_item_number = sorted(list(set(list_item_number)), key=list_item_number.index)
    
    # = = = = = = = = = = = = = = =

    page += 1

    # = = = = = = = = = = = = = = =

    if not html.xpath('//nav[@class="pagination"]/a[@type="next"]'):
        break

len(list_item_number)

1 1
1 2
1 3


484

In [6]:
len(list_item_number) == 0

False

In [7]:
output = pd.DataFrame({'No': [i+1 for i in range(len(list_item_number))],
                       'Item Number': list_item_number})

output = output.sort_values(by=['No'],
                            ascending=[True],
                            ignore_index=True)

output.to_excel('./test_item_number.xlsx', index=False)

output

,No,Item Number
0,1,253139036562
1,2,255922672592
2,3,235653299500
3,4,255586148611
4,5,233621258535
...,...,...
479,480,256202050181
480,481,335398145876
481,482,256081676427
482,483,335397749637


In [8]:
crawler_status = 'ok'

crawler_status

'ok'

In [9]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [10]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./test_item_number_error.xlsx', index=False)

output_error

""
